# Setting up and running analysis on batch runs

In [ ]:
# Install a pip package in the current Jupyter kernel
import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install mesa
!{sys.executable} -m pip install seaborn

In [7]:
import pandas as pd
wolf_batch = pd.read_csv('./seg_model_batch_run_data.csv', index_col=None)
wolf_batch_steps = pd.read_csv('./seg_model_steps_batch_run_data.csv', index_col=None)

### Get a sense of what's happening with your data by doing some digging around:

In [ ]:
# take a peek at the overall structure
wolf_batch.head()

In [ ]:
# take a peek at the overall structure
wolf_batch_steps.head()

### Some thinking about how to work with this and move forward:
We're interested in the relationship between wolves, sheep, and grass. We did a batch run to generate the CSV data (above) that varied the following parameters:

                params = {"grass_regrowth_time": [*range(25,37,5)], 
                        "sheep_gain_from_food": [*range(2,6,1)],
                        "wolf_gain_from_food": [*range(18,28,2)]
                        }

What we will likely want to look at is how these elements come together. We could replicate the plot from a single run, but I don't know how much that will help us progress on answering our question. Instead, we will want to think about creating separate universes where we focus on the variation of one element and how that impacts our findings. 

### Possible research question: How could we think about the relationship between sheep's gain from food and overall sheep population?

In [68]:
# We need to get it on the basis of sheep on run by sheep gain
# generate dataset with just sheep and transpose
keep = ~wolf_batch_steps.columns.isin(['Sheep Gain', 'Wolf Gain', 'Run Num', "Animal"])
sheep = wolf_batch_steps.loc[wolf_batch_steps["Animal"]=="Sheep", keep ].T
sheep.columns = sheep.iloc[0] #rename columns

sheep

Grass Regrowth,25,25,25,25,25,25,25,25,25,25,...,35,35,35,35,35,35,35,35,35,35
Grass Regrowth,25,25,25,25,25,25,25,25,25,25,...,35,35,35,35,35,35,35,35,35,35
0,100,100,100,100,100,100,100,100,100,100,...,100,100,100,100,100,100,100,100,100,100
1,77,75,81,79,83,89,83,80,84,86,...,80,84,89,96,88,88,95,87,86,82
2,67,58,67,64,74,69,73,72,62,68,...,64,74,76,91,74,76,87,77,82,76
3,46,45,55,58,54,58,57,55,44,56,...,57,71,70,81,68,74,81,69,74,70
4,38,34,39,42,42,40,41,36,27,44,...,53,64,67,75,59,69,79,63,66,61
5,28,26,26,38,34,26,34,24,23,30,...,46,57,65,67,56,59,69,60,66,57
6,17,12,15,25,23,20,25,16,15,22,...,39,46,58,59,46,48,62,51,55,47
7,14,10,12,15,19,15,16,9,10,17,...,33,41,47,51,43,41,48,44,51,42
8,11,7,10,9,14,10,10,7,3,11,...,29,39,40,37,40,37,43,38,46,32


In [175]:
from wolf_sheep.model import WolfSheep
from mesa.batchrunner import FixedBatchRunner

import json 
import pandas as pd
from itertools import product

# parameters that will remain constant
fixed_parameters = {
    "height": 20,
    "width": 20,
    "initial_sheep": 100,
    "initial_wolves": 50,
    "sheep_reproduce": 0.04,
    "wolf_reproduce": 0.05,
    "grass": True,
}

# parameters you want to vary
# can also include combinations here
params = {"grass_regrowth_time": [*range(25,37,5)], 
        "sheep_gain_from_food": [*range(2,6,1)],
        "wolf_gain_from_food": [*range(18,27,2)]
        }

# combine all the parameters you want to combine using this function
def dict_product(dicts): #could just use the below but it's cleaner this way
    """
    >>> list(dict_product(dict(number=[1,2], character='ab')))
    [{'character': 'a', 'number': 1},
     {'character': 'a', 'number': 2},
     {'character': 'b', 'number': 1},
     {'character': 'b', 'number': 2}]
    """
    return (dict(zip(dicts, x)) for x in product(*dicts.values()))

parameters_list = [*dict_product(params)]

# what to run and what to collect
# iterations is how many runs per parameter value
# max_steps is how long to run the model
batch_run = FixedBatchRunner(WolfSheep, parameters_list, 
                            fixed_parameters, iterations=10,
                            model_reporters={
                                "Wolves": lambda m: m.wolves,
                                "Sheep": lambda m: m.sheep,
                                "Grass Growth": lambda m: m.grass_growth,
                            }, 
                                max_steps=5)

# run the batches of your model with the specified variations
batch_run.run_all()





600it [00:03, 151.52it/s]


,0,1,2,3
0,25,2,18,0
1,25,2,18,0
2,25,2,18,0
3,25,2,18,1
4,25,2,18,1
...,...,...,...,...
1795,35,5,26,598
1796,35,5,26,598
1797,35,5,26,599
1798,35,5,26,599


In [179]:
############################
# get the data and export it!
#gather our data (dumps into dictionary)
batch_dict_steps = batch_run.get_collector_model()

# IMPORTANT: Think about how you want to use your data
# We're pulling the items into a dataframe and giving things 'nice' names
batch_dict_steps2 = pd.DataFrame([(key,key1,list(val1)) for key,val in batch_dict_steps.items() for key1,val1 in val.items()])
batch_dict_steps2
params = pd.DataFrame(batch_dict_steps2[0].to_list()).rename(columns = {0:"Grass Regrowth", 1:"Sheep Gain", 2:"Wolf Gain"})
#params

animals = pd.DataFrame(batch_dict_steps2[1].to_list()).rename(columns = {1:"Animal"})
animals

,0
0,Wolves
1,Sheep
2,Grass
3,Wolves
4,Sheep
...,...
1795,Sheep
1796,Grass
1797,Wolves
1798,Sheep
